# Data Download

In [65]:
from fredapi import Fred
import pandas as pd
import numpy as np
import statsmodels.api as sm

fred = Fred(api_key='3cca2c46ad523e6d1bb9269895f879fc')

billion = 1000000000
thousand = 1000
 
gdp = fred.get_series('GDPC96')*billion # Real GDP, Billions of Chained 2009 Dollars, Seasonally Adjusted Annual Rate
cons = fred.get_series('PCECC96')*billion # Real Personal Consumption Expenditures, Billions of Chained 2009 Dollars, Seasonally Adjusted Annual Rate
inv = fred.get_series('PNFI')*billion # Private Nonresidential Fixed Investment, Billions of Dollars, Seasonally Adjusted Annual Rate

pop = fred.get_series('B230RC0Q173SBEA')*thousand # Population (midperiod), Thousands, Seasonally Adjusted Annual Rate

non_farm_payroll = fred.get_series('PAYEMS')*thousand # All Employees: Total Nonfarm Payrolls, Thousands of Persons, Seasonally Adjusted
hours_index = fred.get_series("HOANBS") # Nonfarm Business Sector: Hours of All Persons, Index 2009=100, Seasonally Adjusted
average_weekly_hours = fred.get_series("AWHAETP") # Average Weekly Hours of All Employees: Total Private, Monthly

real_hourly_comp_index = fred.get_series("COMPRNFB")# Nonfarm Business Sector: Real Compensation Per Hour, Index 2009=100, Seasonally Adjusted
avg_hourly_comp = fred.get_series("CES0500000003") # Average Hourly Earnings of All Employees: Total Private Dollars per Hour, Seasonally Adjusted

i = fred.get_series('TB3MS') # 3-Month Treasury Bill: Secondary Market Rate, Monthly
cpi = fred.get_series('CPIAUCSL') # Consumer Price Index for All Urban Consumers: All Items, Index 1982-1984=100, Seasonally Adjusted

# Data Processing

In [89]:
def data_processor(data, operation, freq):
    df = pd.DataFrame(np.array(data), index=pd.PeriodIndex(data.index, freq=freq), columns =[str(data)])
    if operation == "mean":
        df = df.groupby(pd.PeriodIndex(data.index, freq=freq), axis=0).mean().to_timestamp().loc["1947-01-01":"2017-01-01"]
    if operation == "sum":
        df = df.groupby(pd.PeriodIndex(data.index, freq=freq), axis=0).sum().to_timestamp().loc["1947-01-01":"2017-01-01"]
    return pd.Series(df[str(data)])

def table_1_replicator(data):
    first_order_autocorr = data.apply(lambda col: col.autocorr(lag=1), axis=0)
    corr = cycle.corr()["Y"]
    df = pd.concat([np.std(data), np.std(data)/np.std(data)["Y"], first_order_autocorr, corr], axis=1)
    df.columns = ["Standard Deviation", "Relative Standard Deviation", "First Order Auto-correlation", "Contemporaneous Correlation with Output"]
    return df 

# Processing the nonfarm payroll time series
payroll = data_processor(non_farm_payroll, "mean", "Q")

# Processing the interest rate series
i_Q = data_processor(i, "mean", "Q")

# Processing the cpi series
cpi_Q = data_processor(cpi, "mean", "Q").pct_change().fillna(0)

# Processing the average weekly hours time series
hours_2009 = float(data_processor(average_weekly_hours["2009-01-01":"2009-12-31"], "mean", "A")*52/4)

# Processing the average hourly earnings time series
avg_hourly_comp_2009 = float(data_processor(avg_hourly_comp["2009-01-01":"2009-12-31"], "mean", "A")*52/4)

# Quarterly Hours = Hours Index(index 2009=100) * Average Quarterly Hours (in 2009) / 100
q_hours = hours_index / 100 * hours_2009 

# Average Real Wage = Real Income / (Average Number of Hours Worked * Number of Workers)
w = real_hourly_comp_index/100 * avg_hourly_comp_2009

# Real Interest Rate = Nominal Interest Rate - Inflation
r = i_Q/100 - cpi_Q

# Creating the data frame
data = pd.concat([gdp/pop, cons/pop, inv["1947-01-01":"2017-01-01"]/pop ,q_hours, (gdp/pop)/q_hours, w, r], axis=1)
data.columns = ["Y","C","I","N","Y/N","w","r"]

# Taking logs
data[["Y","C","I","N","Y/N","w"]] = np.log(data[["Y","C","I","N", "Y/N","w"]])

# Detrending with HP filter
cycle, trend = sm.tsa.filters.hpfilter(data, lamb=1600)
#cycle, trend = sm.tsa.filters.hpfilter(data["1947-01-01":"1996-12-31"], lamb=1600) # For comparing with the paper results

# Computing the first order detrenting
diff = data.diff(periods=1, axis=0).fillna(0)


# Results

In [90]:
# Replication of Table 1
table_1 = table_1_replicator(cycle*100).round(decimals=2)
table_1

,Standard Deviation,Relative Standard Deviation,First Order Auto-correlation,Contemporaneous Correlation with Output
Y,1.62,1.00,0.85,1.00
C,1.26,0.77,0.82,0.77
I,4.75,2.93,0.88,0.67
N,1.89,1.16,0.91,0.87
Y/N,0.92,0.57,0.76,-0.02
w,0.96,0.59,0.68,0.15
r,1.02,0.63,0.71,0.24


In [82]:
# Replication of Table 1 with first order difference (without HP Filter)
table_1_diff = table_1_replicator(diff*100).round(decimals=2)
table_1_diff

,Standard Deviation,Relative Standard Deviation,First Order Auto-correlation,Contemporaneous Correlation with Output
Y,0.94,1.00,0.37,1.00
C,0.81,0.87,0.09,0.77
I,2.50,2.66,0.50,0.67
N,0.89,0.95,0.63,0.87
Y/N,0.67,0.71,0.06,-0.02
w,0.81,0.86,-0.06,0.15
r,0.80,0.86,-0.00,0.24
